StepFun 图片理解最佳实践（Jupyter 笔记）

本笔记基于 StepFun 官方文档“图片理解最佳实践”，整理了常用用法与示例代码，方便快速上手与复用。

参考文档：
- [图片理解最佳实践](https://platform.stepfun.com/docs/guide/image_chat)

内容涵盖：
- 简单图片理解（传入单张图片）
- 多轮对话与多图描述策略
- `detail` 参数的使用（low/high）
- 图片优化（resize、压缩）以降低首字延时
- 透明 PNG 适配（RGBA 转 RGB 白底）
- 以 Base64 方式传图
- 常见问题与限制说明

> 目前推荐使用 step-1o-turbo-vision 模型。该模型拥有最强的视频理解能力，推荐默认开启 detail 模式。


In [ ]:
# 环境依赖与客户端初始化
# - 需要 openai>=1.30.0 （API 兼容 StepFun）
# - 需要设置环境变量 API_KEY

import os
from openai import OpenAI

API_KEY = os.getenv("STEPFUN_API_KEY")
if not API_KEY:
    raise RuntimeError("未检测到环境变量 API_KEY，请在运行前设置，例如：export API_KEY='sk-xxx'")

# 指向 StepFun 平台
client = OpenAI(api_key=API_KEY, base_url="https://api.stepfun.com/v1")

DEFAULT_VISION_MODEL = "step-1o-turbo-vision"
# TEST_IMAGE_URL = "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
TEST_IMAGE_URL ="https://gd-hbimg.huaban.com/14eb4eb2e87b9a46973e9ddca872750043375ad02818c-ildFmT_fw1200webp"
system_prompt = "你是由阶跃星辰提供的AI聊天助手，你除了擅长中文，英文，以及多种其他语言的对话以外，还能够根据用户提供的图片，对内容进行精准的内容文本描述。在保证用户数据安全的前提下，你能对用户的问题和请求，作出快速和精准的回答。同时，你的回答和建议应该拒绝黄赌毒，暴力恐怖主义的内容"



Client initialized for StepFun.


In [ ]:
# 简单图片理解示例
# 传入一张图片并请求描述
# 阶跃星辰支持在 image_url 类型中使用 URL 或 Base64 格式的内容，为了保证更好的性能，推荐使用 URL 来完成图片参数的传递

completion = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "用优雅的语言描述这张图片"},
                {
                    "type": "image_url",
                    "image_url": {"url": TEST_IMAGE_URL},
                },
            ],
        },
    ],
)
print(completion)
print(completion.choices[0].message.content)


ChatCompletion(id='f0e922582aeb5388b18ad4673f612111.44efea37843e07c2df3872eb6a21de5e', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这张图片展现了一位身着优雅礼服的女性，她穿着一件深蓝色丝绒吊带长裙，裙身光滑细腻，尽显高贵与神秘。礼服外层搭配了一层浅蓝色薄纱材质的披肩，轻盈飘逸，为整体造型增添了一丝柔美与梦幻感。她的长发自然垂落，微微卷曲，散发出一种温婉动人的气质。背景是绿意盎然的花园，模糊的花卉与树木营造出一种宁静而唯美的氛围，与她的优雅相得益彰。整体画面宛如一幅精致的油画，充满了诗意与浪漫。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1757587616, model='step-1o-turbo-vision', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=133, prompt_tokens=266, total_tokens=399, completion_tokens_details=None, prompt_tokens_details=None))
这张图片展现了一位身着优雅礼服的女性，她穿着一件深蓝色丝绒吊带长裙，裙身光滑细腻，尽显高贵与神秘。礼服外层搭配了一层浅蓝色薄纱材质的披肩，轻盈飘逸，为整体造型增添了一丝柔美与梦幻感。她的长发自然垂落，微微卷曲，散发出一种温婉动人的气质。背景是绿意盎然的花园，模糊的花卉与树木营造出一种宁静而唯美的氛围，与她的优雅相得益彰。整体画面宛如一幅精致的油画，充满了诗意与浪漫。


In [8]:
# 多图较多场景：先生成描述再继续对话
# 当图片数量可能超过上限（当前单次请求上限为 60 张）时，可先用模型为每张图生成描述
# 然后将描述作为对话上下文，继续进行真正的问答。
# step-1v 默认会选择低分辨率，每张图片 400 token
# step-1o 系列模型低分辨率情况下，默认每张图片 169 token；当 detail 模式为 high 时，图片的 Token 消耗将会基于图片大小进行计算


def describe_image(img_url: str, detail: str = "high") -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "请详细描述这张图片中的内容"},
                {"type": "image_url", "image_url": {"url": img_url, "detail": detail}},
            ],
        },
    ]
    completion = client.chat.completions.create(model=DEFAULT_VISION_MODEL, messages=messages)
    print(completion)
    return completion.choices[0].message.content

# 示例：对单张图先生成描述，然后把描述接入真正对话
context_messages = []
image_desc_low = describe_image(TEST_IMAGE_URL, detail="low")
print("low detail:",image_desc_low)
image_desc_high = describe_image(TEST_IMAGE_URL, detail="high")
print("high detail:",image_desc_high)
# context_messages.append({"role": "user", "content": image_desc})

# print("生成的图片描述（截断预览）：\n", image_desc[:300], "...")


ChatCompletion(id='317d8951e4ba2be492501cec49f20737.aff43aad13aabfe3982fd98c2fd08e32', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这是一张女性的正面肖像照片，整体风格优雅大方。以下是图片的详细描述：\n\n### 1. **人物姿态与表情**：\n   - 女性面向镜头，表情端庄大方，眼神直视前方，显得自信且从容。\n   - 她的头发是深色长发，发型是半扎的样式，上半部分头发被整齐地束在头顶，下半部分是自然垂落的波浪卷发，整体发型显得优雅且有女人味。\n\n### 2. **服饰与装扮**：\n   - 她穿着一件黑色的礼服，礼服的设计是抹胸款式，突显出优雅的肩部线条。\n   - 礼服的上半部分有独特的设计，看起来材质可能有所不同，左侧（图片的右侧）似乎有拼接的设计，增加了服装的层次感。\n   - 她的右手（图片的左侧）手指上戴着一枚显眼的戒指，戒指设计精致，与整体装扮相得益彰。\n   - 她的脖子上戴着一条闪亮的钻石项链，项链设计精美，增添了高贵的气息。\n   - 她的耳朵上戴着一对大型的耳环，耳环造型复杂，呈流苏状，带有钻石装饰，在光线下闪耀，非常吸睛。\n\n### 3. **场景与背景**：\n   - 背景是室内的环境，背景颜色主要是红色，右侧边缘可以看到一部分白色的物体，可能是沙发或座椅。\n   - 女性坐在一个地方，膝盖上盖着一块浅色的毛绒织物，织物看起来非常柔软，可能是为了保暖或装饰。\n   - 她的左耳边有一个小型的麦克风设备，这表明她可能处于一个活动或访谈的场合。\n\n### 4. **整体风格**：\n   - 整体造型非常优雅、端庄，适合正式场合，例如颁奖典礼、时尚活动或高端访谈。\n   - 从她的妆容和打扮来看，她非常注重细节，整体给人一种精致、高贵的感觉。\n\n### 5. **氛围**：\n   - 图片的色调偏暖，红色背景与她的黑色礼服形成鲜明对比，使她成为画面的焦点。\n   - 光线柔和地打在她的脸上和身上，突显出她的面部轮廓和服装的质感。\n\n这张图片很可能是在一场

In [ ]:
# 多图场景
#  TEST_IMAGE_URL = "https://q0.itc.cn/q_70/images01/20240612/8e0b7aecb4984be19faaa78f4ecd7c92.jpeg"
# "https://gd-hbimg.huaban.com/2b4d0f9fc1515d4d1d2d498b8318dbac3491cc58175ee-83PEcb_fw1200webp"
# "https://gd-hbimg.huaban.com/30576c5c513d28154df3cee1570027bb95a47bc0413d4-6sNdZc_fw1200webp"

In [ ]:
# 预处理：对图片缩放与压缩来获得较好的处理速度
# - step-1o 对于 detail=low 或默认，可将最长边缩放至 728px
# - step-1o 对于 detail=high，可将最长边缩放至 504 的倍数（示例用 2688）
# - step-1v 对于 detail=low 或默认，可将最长边缩放至 1280px
# - step-1v 对于 detail=high，可将最长边缩放至 2688px

# 4k图片
# https://images.pexels.com/photos/18899626/pexels-photo-18899626.jpeg
# https://images.pexels.com/photos/1146134/pexels-photo-1146134.jpeg

def resize_image(input_path: str, output_path: str, max_size: int) -> None:
    image = Image.open(input_path)
    width, height = image.size
    if width > height:
        new_width = max_size
        new_height = int((max_size / width) * height)
    else:
        new_height = max_size
        new_width = int((max_size / height) * width)
    resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
    resized_image.save(output_path)


def compress_image(input_path: str, output_path: str, quality: int = 80) -> None:
    image = Image.open(input_path)
    image.save(output_path, quality=quality)

print("已定义 resize_image 与 compress_image 实用函数。")


已定义 resize_image 与 compress_image 实用函数。


In [ ]:
# 透明 PNG 适配：RGBA 转 RGB 白色背景
# https://gd-hbimg.huaban.com/7c4795bc026c359c8d17ff19d1d25b574fbb43e03088e9-ena8d7_fw1200webp
from PIL import Image

def convert_rgba_to_rgb_with_white_background(input_path: str, output_path: str) -> None:
    img = Image.open(input_path)
    if img.mode != "RGBA":
        raise ValueError("输入图片不是 RGBA 模式")
    white_background = Image.new("RGB", img.size, (255, 255, 255))
    white_background.paste(img, mask=img.split()[3])
    result = white_background.convert("RGB")
    result.save(output_path)

print("已定义 convert_rgba_to_rgb_with_white_background 实用函数。")


已定义 convert_rgba_to_rgb_with_white_background 实用函数。


In [7]:
# detail 参数演示：low 与 high

def ask_with_detail(img_url: str, detail: str):
    return client.chat.completions.create(
        model=DEFAULT_VISION_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"detail={detail}：请概述这张图片的内容"},
                    {"type": "image_url", "image_url": {"url": img_url, "detail": detail}},
                ],
            },
        ],
    ).choices[0].message.content

print("low 模式：\n", ask_with_detail(TEST_IMAGE_URL, "low")[:400], "...\n")
print("high 模式：\n", ask_with_detail(TEST_IMAGE_URL, "high")[:400], "...\n")


low 模式：
 这张图片展示了一位女性，她穿着一件黑色的抹胸礼服，整体造型优雅大方。她的头发是长卷发，部分头发编成辫子并别在脑后，其余头发自然垂落。她佩戴着一对精致的银色耳环，耳环设计复杂，带有花纹装饰。脖子上戴着一条银色的项链，项链设计独特，与耳环相呼应。她的手指上还戴着一枚戒指，整体饰品搭配显得高贵典雅。背景是红色和白色的搭配，营造出一种正式场合的氛围。 ...

high 模式：
 这张图片展示了一位穿着黑色礼服的女性，她坐在一个背景为红色和米色的环境中。她的头发是黑色的，呈波浪状，部分头发编成辫子并别在头顶，其余的头发自然垂落。她佩戴着一对精致的长款耳环，耳环设计复杂，带有钻石装饰，显得非常华丽。她还佩戴了一条与耳环相配的钻石项链，项链设计优雅，突显了整体的高贵感。她的右手戴着一枚钻石戒指，整体造型非常优雅大方。她身着一件黑色无肩带礼服，礼服的设计简洁而现代。她的姿态端庄，双手交叠放在膝盖上，手上搭着一块浅色毛绒布料，整体给人一种优雅、从容的感觉。 ...



In [8]:
# 以 Base64 输入图片
import base64
import requests


def url_to_base64(image_url: str) -> str:
    resp = requests.get(image_url, timeout=20)
    resp.raise_for_status()
    return base64.b64encode(resp.content).decode("utf-8")


b64_content = url_to_base64(TEST_IMAGE_URL)

completion_b64 = client.chat.completions.create(
    model=DEFAULT_VISION_MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "使用 Base64 输入，请描述图片"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/webp;base64,{b64_content}",
                        "detail": "low",
                    },
                },
            ],
        },
    ],
)

print(completion_b64.choices[0].message.content[:500], "...")


The image shows a person wearing an elegant black outfit with a strapless design. The person has long, wavy, dark hair styled with some sections pulled back and adorned with a dark accessory. They are wearing intricate, large earrings with a floral or leaf-like design, along with a detailed silver necklace that has a prominent pendant. A silver ring is visible on one of their fingers. The background features a warm, red tone, and there is a light-colored object, possibly fabric, in the person's  ...


## 常见问题、限制与参考

- **图片格式支持**：JPG/JPEG、PNG、静态 GIF、WebP；通过 URL 或 Base64 传参。
- **单次请求图片数量上限**：当前为 60 张。如超出，建议先逐张生成描述，再以文本上下文继续对话。
- **指令位置建议**：多图场景将指令置于尾部、图片置于前部，提升指令跟随。
- **优化首字延时**：
  - `detail=low` 或默认：最长边约 728px；
  - `detail=high`：最长边可用 504 的倍数（如 2688），配合质量 `quality≈80` 压缩；
- **透明 PNG**：透明通道可能视为黑色，可先转白底。

参考：
- StepFun 官方文档 · 图片理解最佳实践（`https://platform.stepfun.com/docs/guide/image_chat`）
